# Imports

In [3]:
import ccxt
from datetime import datetime
import psycopg2
from dotenv import find_dotenv, dotenv_values

# Config / Params

In [4]:
config = dotenv_values(find_dotenv())

user = config.get('USERNAME_PG')
password = config.get('PASSWORD_PG')

# Helper Functions

In [5]:
def insert_ohlc_data(db_params, table_name, exchange, symbol, ohlc_data):
    """
    Inserts OHLC data into a dynamically specified PostgreSQL table.

    Parameters:
    - db_params: A dictionary containing database connection parameters.
    - table_name: The name of the table to insert data into.
    - exchange: The name of the exchange (e.g., 'Binance').
    - symbol: The trading pair or symbol (e.g., 'BTCUSDT').
    - ohlc_data: The OHLC data from the Binance API.
    """
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    # SQL query to insert data, using safe string formatting for the table name
    insert_query = f"""
    INSERT INTO {table_name} (exchange, symbol, datetime, open, high, low, close, base_volume, quote_volume, base_buy_volume, quote_buy_volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for data in ohlc_data:
        # Convert timestamp to datetime
        datetime_val = datetime.utcfromtimestamp(int(data[0]) / 1000.0)

        # Prepare data tuple
        data_tuple = (
            exchange,
            symbol,
            datetime_val,
            data[1],  # open
            data[2],  # high
            data[3],  # low
            data[4],  # close
            data[5],  # base_volume
            data[7],  # quote_volume (index 6 is close time, so we use index 7)
            data[9],  # base_buy_volume
            data[10]  # quote_buy_volume
        )

        # Execute the insert query
        cur.execute(insert_query, data_tuple)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

# Get Data From CCXT

In [6]:
# Initialize the Binance Futures exchange
bnf_client = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})

In [27]:
exchange_name = 'binance'
symbol = 'BTCUSDT'

ohlc_data = bnf_client.fapipublic_get_klines(params={'symbol':symbol, 'interval':'1m', 'limit':10, 'startTime':0})

# Write Values to Database

In [54]:
# Example usage
db_params = {
    'dbname': 'crypto_data',
    'user': user,
    'password': password,
    'host': 'localhost'
}
table_name = 'ohlc_1m_swaps_test'

In [57]:
insert_ohlc_data(db_params, table_name, exchange_name, symbol, ohlc_data)

In [58]:
import pandas as pd

pd.DataFrame(ohlc_data)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1567965420000,10000,10000,10000,10000,0.001,1567965479999,10,1,0,0,0
1,1567965480000,10000,10000,10000,10000,0,1567965539999,0,0,0,0,0
2,1567965540000,10000,10000,10000,10000,0.001,1567965599999,10,1,0.001,10,0
3,1567965600000,10000,10000,10000,10000,0,1567965659999,0,0,0,0,0
4,1567965660000,10000,10000,10000,10000,0,1567965719999,0,0,0,0,0
5,1567965720000,10000,10000,10000,10000,0,1567965779999,0,0,0,0,0
6,1567965780000,10000,10000,10000,10000,0,1567965839999,0,0,0,0,0
7,1567965840000,10000,10000,10000,10000,0,1567965899999,0,0,0,0,0
8,1567965900000,10000,10000,10000,10000,0,1567965959999,0,0,0,0,0
9,1567965960000,10000,10000,10000,10000,0,1567966019999,0,0,0,0,0


# Time Test

In [9]:
import time
import requests

## Through CCXT

In [7]:
start_time = time.time()
print('making call to Binance API through ccxt')
symbol = 'BTCUSDT'
ohlc_data = bnf_client.fapipublic_get_klines(params={'symbol':symbol, 'interval':'1m', 'limit':1500, 'startTime':0})
print(f'Data retrieved in {(time.time() - start_time):.2f}s')

making call to Binance API through ccxt
Data retrieved in 1.40s


## Direct Through API

In [10]:
endpoint = 'https://fapi.binance.com/fapi/v1/klines'
headers = {
    'accept':'application/json',
}
params = {
    'symbol':symbol,
    'interval':'1m',
    'limit':1500,
    'startTime':0,
}

start_time = time.time()
print('making call to Binance API with requests')
response = requests.get(endpoint, headers=headers, params=params)
print(f'Data retrieved in {(time.time() - start_time):.2f}s')

making call to Binance API with requests
Data retrieved in 1.40s


In [13]:
response.json()

[[1567965420000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0.001',
  1567965479999,
  '10',
  1,
  '0',
  '0',
  '0'],
 [1567965480000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965539999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965540000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0.001',
  1567965599999,
  '10',
  1,
  '0.001',
  '10',
  '0'],
 [1567965600000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965659999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965660000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965719999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965720000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965779999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965780000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965839999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965840000,
  '10000',
  '10000',
  '10000',
  '10000',
  '0',
  1567965899999,
  '0',
  0,
  '0',
  '0',
  '0'],
 [1567965900000,
  '10000',
  '10